In [167]:
import pandas as pd
import pickle
from sklearn.ensemble import AdaBoostRegressor
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error

In [168]:
# Use only last 3 datasets for fitting models

#df1 = pd.read_csv('datasets/db_waw_flats.csv', parse_dates = ['today', 'publication_date'])
#df2 = pd.read_csv('datasets/db_waw_flats_17_06.csv', parse_dates = ['today', 'publication_date'])
#df3 = pd.read_csv('datasets/db_waw_flats_27_6.csv', parse_dates = ['today', 'publication_date'])
#df4 = pd.read_csv('datasets/db_waw_flats_10_7.csv', parse_dates = ['today', 'publication_date'])
#df5 = pd.read_csv('datasets/db_waw_flats_27_7.csv', parse_dates = ['today', 'publication_date'])
#df6 = pd.read_csv('datasets/db_waw_flats_3_8.csv', parse_dates = ['today', 'publication_date'])
#df7 = pd.read_csv('datasets/db_waw_flats_16_8.csv', parse_dates = ['today', 'publication_date'])
df7 = pd.read_csv('datasets/db_waw_flats_10_9.csv', parse_dates = ['today', 'publication_date'])
df8 = pd.read_csv('datasets/db_waw_flats_14_9.csv', parse_dates = ['today', 'publication_date'])
df9 = pd.read_csv('datasets/db_waw_flats_25_9.csv', parse_dates = ['today', 'publication_date'])

In [169]:
common_df = pd.concat([df7,df8,df9], ignore_index=True)
common_df

,price,area,rooms,renovation,floor,market,elevator,street,district,balcony,terrace,garden,parking,central_heating,seller,blok,cena_m,today,publication_date
0,1395000.0,77.70,3.0,3,0,2,0,stary żoliborz,żoliborz,0,0,0,0,1,2,0,17953.667954,2023-09-10,2023-07-28 19:32:05
1,1400000.0,82.00,3.0,3,2,2,1,wierzbno,mokotów,1,0,0,1,1,1,0,17073.170732,2023-09-10,2023-08-16 23:15:39
2,850000.0,56.90,2.0,0,2,2,0,bródno-podgrodzie,targówek,0,0,0,0,0,1,0,14938.488576,2023-09-10,2023-07-14 18:36:04
3,599000.0,36.10,2.0,3,2,2,1,chomiczówka,bielany,0,0,0,1,1,1,1,16592.797784,2023-09-10,2023-02-08 20:46:01
4,830000.0,57.00,3.0,3,2,1,1,unknown,ursus,1,0,0,1,1,2,1,14561.403509,2023-09-10,2023-08-01 16:07:41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22725,1479400.0,113.80,4.0,3,1,2,1,ul. meander,ursynów,1,0,0,1,1,1,1,13000.000000,2023-09-25,2015-10-27 20:02:18
22726,NaN,319.00,2.0,3,2,2,0,stara miłosna,wesoła,0,0,0,1,0,1,0,NaN,2023-09-25,2015-02-06 23:10:16
22727,NaN,262.00,6.0,3,1,2,0,stary żoliborz,żoliborz,0,0,0,1,0,1,0,NaN,2023-09-25,2015-02-06 22:28:39
22728,2700000.0,200.00,8.0,1,1,2,0,ul. łotewska,praga-południe,0,0,0,0,0,1,0,13500.000000,2023-09-25,2014-11-10 12:05:12


In [170]:
common_df.drop(columns=['today']).drop_duplicates(keep='last', inplace=True)

In [171]:
cols_x = ['district', 'area', 'rooms', 'renovation', 'floor', 'balcony', 'terrace',
          'garden', 'parking', 'central_heating', 'market', 'seller', 'blok', 'elevator']

cat_cols = ['district', 'renovation', 'floor', 'balcony', 'terrace', 'garden', 'parking',
            'central_heating', 'market', 'seller', 'blok', 'elevator']
col_y = ['price']

model_df = common_df[cols_x + col_y].dropna()

In [172]:
#le = preprocessing.LabelEncoder()
#le.fit(model_df['district'])
le = pickle.load(open('district_encode', 'rb'))
model_df['district'] = le.transform(model_df['district'])

In [173]:
X_train, X_test, y_train, y_test = train_test_split(model_df[cols_x], model_df['price'], test_size=0.2, shuffle=False)

In [174]:
from sklearn.tree import DecisionTreeRegressor
est = DecisionTreeRegressor(random_state=0, max_depth=12)
ada_model = AdaBoostRegressor(estimator=est, random_state=0, n_estimators=50, learning_rate=0.05, loss='square')
ada_model.fit(X_train, y_train)
ada_preds = ada_model.predict(X_test)

In [175]:
# Adaboost quality
print('percentage error', mean_absolute_percentage_error(y_test, ada_preds))
print('absolute error', mean_absolute_error(y_test, ada_preds))
print('root squared error', mean_squared_error(y_test, ada_preds, squared=False))

percentage error 0.08163336520029016
absolute error 80323.24765735304
root squared error 138322.23844322146


In [117]:
# Catboost is fitting automatically during parsing new data in parse_data.py

#from catboost import Pool, CatBoostRegressor
#cat_df = pd.concat([df6,df7,df8], ignore_index=True)
#cat_df.drop(columns=['today']).drop_duplicates(keep='last', inplace=True)
#cat_df = cat_df[cols_x + col_y].dropna()
#cat_df['district'] = le.transform(cat_df['district'])

#X_train, X_test, y_train, y_test = train_test_split(model_df[cols_x], model_df['price'], test_size=0.25, random_state=10)
#train_pool = Pool(X_train, y_train, cat_features=cat_cols)
#test_pool = Pool(X_test, cat_features=cat_cols)
#cb_model = CatBoostRegressor(iterations=3000, learning_rate=0.04, l2_leaf_reg=1, depth=8)
#cb_model.fit(train_pool)
#cb_preds = cb_model.predict(test_pool)

In [176]:
# Load catboost
cb_model = CatBoostRegressor().load_model("cb_model.cbm")

test_pool = Pool(X_test, cat_features=cat_cols)
cb_preds = cb_model.predict(test_pool)

In [177]:
print('percentage error', mean_absolute_percentage_error(y_test, cb_preds))
print('absolute error', mean_absolute_error(y_test, cb_preds))
print('root squared error', mean_squared_error(y_test, cb_preds, squared=False))

percentage error 0.08833748111316926
absolute error 105064.15419122363
root squared error 229691.02895550028


In [178]:
lr_df = pd.DataFrame(data=ada_preds.reshape(-1,1), columns=['ada_preds'])
lr_df['cb_preds'] = cb_preds
lr_df['y'] = y_test.values
train = lr_df[:2500]
test = lr_df[2500:]

In [179]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train.drop(columns=['y']), train['y'])
lr_preds = lr.predict(test.drop(columns=['y']))

In [180]:
# Check regression quality
print('percentage error', mean_absolute_percentage_error(test['y'], lr_preds))
print('absolute error', mean_absolute_error(test['y'], lr_preds))
print('root squared error', mean_squared_error(test['y'], lr_preds, squared=False))

percentage error 0.08341447093701203
absolute error 93913.3620412701
root squared error 146306.4014600447


In [181]:
# Fit models on full dataset
ada_model.fit(model_df[cols_x], model_df['price'])
ada_preds = ada_model.predict(model_df[cols_x])
cb_preds = cb_model.predict(Pool(model_df[cols_x], cat_features=cat_cols))
new_df = pd.DataFrame(data=ada_preds.reshape(-1,1), columns=['ada_preds'])
new_df['cb_preds'] = cb_preds
new_df['y'] = model_df['price']
lr.fit(new_df.drop(columns=['y']), model_df['price'])

LinearRegression()

In [182]:
# save models to file
import pickle
filename = 'adaboost_regressor.pkl'
pickle.dump(ada_model, open(filename, 'wb'))
#pickle.dump(le, open('district_encode', 'wb'))
filename = 'linear_regressor.pkl'
pickle.dump(lr, open(filename, 'wb'))
#cb_model.save_model('cb_model.cbm')